In [1]:
import math
from scipy.optimize import minimize
import numpy as np

In [2]:
### tools：

def minfunction(f, x0, bounds):
    """
    求一个函数在边界内的最小值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    res = minimize(f, x0, method='SLSQP', bounds=bounds)
    return res

def maxfunction(f, x0, bounds):
    """
    求一个函数在边界内的最大值
    f：函数
    x0：初始值
    bounds：边界条件。[(min, max),...]
    """
    fun = lambda X : -f(X)
    res = minimize(fun, x0, method='SLSQP', bounds=bounds)
    return res


In [3]:
### 目标函数：
bounds = [(0.001,10),(0.001,10),(0.001,10),(0.001,10),(0.001,10),(0.001,10),(0.001,10),(0.001,10),(0.001,10),(0.001,10)]
X_optimal = [0.0406684113216282, 0.147721240492452, 0.783205732104114,0.00141433931889084, 
             0.485293636780388, 0.000693183051556082, 0.0274052040687766,0.0179509660214818, 0.0373268186859717, 0.0968844604336845]
y_optimal = -47.7648884594915
def G14(X):
    c = [-6.089, -17.164, -34.054, -5.914, -24.721, -14.986, -24.1, -10.708, -26.662, -22.179]
    y = np.sum([X[i]*(c[i]+np.log(X[i]/(X[0]+X[1]+X[2]+X[3]+X[4]+X[5]+X[6]+X[7]+X[8]+X[9]))) for i in range(10)])
    h1 = X[0]+2*X[1]+2*X[2]+X[5]+X[9]-2-0.0001
    h11 = -X[0]-2*X[1]-2*X[2]-X[5]-X[9]+2-0.0001
    h2 = X[3]+2*X[4]+X[5]+X[6]-1-0.0001
    h22 = -X[3]-2*X[4]-X[5]-X[6]+1-0.0001
    h3 = X[2]+X[6]+X[7]+2*X[8]+X[9]-1-0.0001
    h33 = -X[2]-X[6]-X[7]-2*X[8]-X[9]+1-0.0001
    return y,h1,h11,h2,h22,h3,h33

In [4]:
# y = lambda X: G14(X)[0]
# x0 = [5,5,5,5,5,5,5,5,5,5]
# x0 = [10,10,10,10,10,10,10,10,10,10]

In [5]:
# minfunction(y,x0,bounds)

In [6]:
### 适应度函数
def fitnessG14_num(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = 1220.174
    fmin = 0.008509619375000003
    count = np.sum(np.array(problem1(X))[1:5]>0)
    
    return -(problem1(X)[0]+epsion*(fmax-fmin)*count)  

def fitnessG14_vio(X):
    """
    problem1关于数量惩罚的适应度函数
    X: 对应的解
    epsion：惩罚系数大于fmax-fmin的倍数
    """
    fmax = -0.21
    fmin = -2096.0285092966406
    vmax = 176
    e = 0.1
    n = 6
    
    res = G14(X)
    fitness = -res[0]
    count = np.sum(np.array(res)[1:7]>0)
    
    
    if count > 0:
        for i in range(1,len(res)):
            if res[i] > 0:
                cathy = (fmax - fmin)*(e*vmax*n/count+res[i])/(e*vmax)
                fitness = fitness - cathy
    return fitness    


In [7]:
### PSO
class PSO_fit():
    def __init__(self, pN, dim, max_iter, bounds):
        self.wmax = 0.8 #最大的惯性权重
        self.wmin = 0.4 #最小的惯性权重
        self.c1 = 2  #加速因子
        self.c2 = 2  #加速因子
        self.pN = pN  # 粒子数量
        self.dim = dim  # 搜索维度
        self.max_iter = max_iter  # 迭代次数
        self.bounds = bounds
        
        self.X = np.zeros((self.pN, self.dim))  # 所有粒子的位置：一个pN*dim的矩阵
        self.V = np.zeros((self.pN, self.dim))  # 左右粒子的速度：一个pN*dim的矩阵
        self.pbest = np.zeros((self.pN, self.dim))  # 个体经历的最佳位置：一个pN*dim的矩阵
        self.gbest = np.zeros((self.pN, self.dim))      # 全局最佳位置：一个pN*dim的矩阵
        self.p_fit = np.full(self.pN,-10.0e+28)  # 每个个体的历史最佳适应值
        self.g_fit = np.full(self.pN,-10.0e+28)    # 全局最佳适应值
    
    def init_Population(self):
        """
        初始化种群
        :return:
        """
        
        for i in range(self.pN):
            for j in range(self.dim):
                self.X[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
                self.V[i][j] = self.bounds[j][0] + (self.bounds[j][1] - self.bounds[j][0]) * np.random.random()
        for i in range(self.pN):
            tmp = fitnessG14_vio(self.X[i])
            if (tmp > self.p_fit[i]):
                self.p_fit[i] = tmp
                self.pbest[i] = self.X[i]
                
        for i in range(self.pN):
            indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/100))
            for j in indexes:
                if self.p_fit[j] > self.g_fit[i]:
                    self.g_fit[i] = self.p_fit[j]
                    self.gbest[i] = self.pbest[j]

#             self.pbest[i] = self.X[i]
#             tmp = fitnessG01_num(self.X[i])
#             self.p_fit[i] = tmp
#             if (tmp > self.fit):
#                 self.fit = tmp
#                 self.gbest = self.X[i]
        
    def iterator(self):
        """
        迭代
        """
        for t in range(self.max_iter):
            w = self.wmin + (self.wmax - self.wmin) * np.exp(- (4 * t / self.max_iter) * (4 * t / self.max_iter))
            
            for i in range(self.pN):
                self.V[i] = w*self.V[i]+self.c1*np.random.random()*(self.pbest[i]-self.X[i])+self.c2*np.random.random()*(self.gbest[i]-self.X[i])
                self.X[i] = self.X[i] + self.V[i]
                
                """
                控制在搜索范围内
                """ 
                for j in range(self.dim):
                    if self.X[i][j] < self.bounds[j][0]:
                        self.X[i][j] = self.bounds[j][0]
                    if self.X[i][j] > self.bounds[j][1]:
                        self.X[i][j] = self.bounds[j][1]     
            for i in range(self.pN):
                tmp = fitnessG14_vio(self.X[i])
                if (tmp > self.p_fit[i]):
                    self.p_fit[i] = tmp
                    self.pbest[i] = self.X[i]
            for i in range(self.pN):
                indexes = np.random.randint(low=0,high=self.pN,size=int(self.pN/100))
                for j in indexes:
                    if self.p_fit[j] > self.g_fit[i]:
                        self.g_fit[i] = self.p_fit[j]
                        self.gbest[i] = self.pbest[j]
            print(self.g_fit)  
        return self.gbest, self.g_fit

In [8]:
PSO = PSO_fit(pN=4000, dim=10, max_iter=1000, bounds=bounds)

In [9]:
PSO.init_Population()

In [10]:
PSO.iterator()

[-15178.86502178 -17210.96472196 -16922.60028808 ... -17153.6582236
 -17173.89967783 -17230.14776522]
[-13170.00792649 -13048.84466648 -13751.58009984 ... -13491.99410267
 -12929.60636772 -13837.25733567]
[-12764.98664574 -12769.66308332 -12785.36237482 ... -12838.8002394
 -12690.08113812 -12765.61891435]
[-12764.98664574 -12703.2323364  -12695.32837099 ... -12771.89708098
 -12667.82056659 -12723.22135282]
[-12755.15632289 -12668.72404938 -12695.32837099 ... -12679.91766706
 -12621.86672729 -12626.79821506]
[-12691.09466487 -12668.72404938 -12621.86672729 ... -12679.91766706
 -12621.86672729 -12626.79821506]
[-12554.51434256 -12664.45454556 -12621.86672729 ... -12664.45454556
 -12621.86672729 -12626.79821506]
[-12554.51434256 -12625.09715216 -12621.86672729 ... -12613.94960056
 -12615.37798089 -12626.79821506]
[-12554.51434256 -12601.20439844 -12595.44810543 ... -12578.44869127
 -12539.76649024 -12593.29895162]
[-12554.51434256 -12554.51434256 -12566.33362509 ... -12548.17557202
 -1253

[-12532.2581297  -12532.23051697 -12532.29011349 ... -12532.22147231
 -12532.24564964 -12532.28577991]
[-12532.2581297  -12532.23051697 -12532.29011349 ... -12532.22147231
 -12532.19972441 -12532.28577991]
[-12532.20446413 -12532.23051697 -12532.29011349 ... -12532.22147231
 -12532.19972441 -12532.19356252]
[-12532.20446413 -12532.23051697 -12532.29011349 ... -12532.22147231
 -12532.19972441 -12532.19356252]
[-12532.20446413 -12532.23051697 -12532.29011349 ... -12532.22147231
 -12532.19972441 -12532.19356252]
[-12532.20179094 -12532.23051697 -12532.22265402 ... -12532.22147231
 -12532.19972441 -12532.19356252]
[-12532.20179094 -12532.20735899 -12532.22265402 ... -12532.22147231
 -12532.19972441 -12532.19356252]
[-12532.20179094 -12532.20735899 -12532.22265402 ... -12532.20735899
 -12532.19972441 -12532.19356252]
[-12532.20179094 -12532.20735899 -12532.22265402 ... -12532.20735899
 -12532.19972441 -12532.19356252]
[-12532.19042546 -12532.20735899 -12532.22265402 ... -12532.20735899
 -12

[42.9511439  42.94894776 42.95736341 ... 42.97487267 42.92103509
 42.93932125]
[42.9511439  42.97054717 42.9707687  ... 42.97487267 42.92103509
 42.96842375]
[42.95331746 42.97054717 42.9707687  ... 42.97487267 42.98320964
 42.97036911]
[42.95449679 42.97054717 42.9707687  ... 42.97487267 42.98320964
 42.97137013]
[42.95449679 42.98074376 42.9707687  ... 42.97487267 42.98320964
 42.98074376]
[42.98162563 42.98074376 42.97408649 ... 42.97487267 42.98320964
 42.98074376]
[42.98162563 42.98074376 42.97487267 ... 42.97633193 42.98320964
 42.98074376]
[42.98162563 42.98074376 42.97487267 ... 42.983481   42.98320964
 42.98074376]
[42.98162563 42.98074376 42.97487267 ... 42.983481   42.98320964
 42.98074376]
[42.98162563 42.98519302 42.97487267 ... 42.983481   42.98320964
 42.98074376]
[42.98162563 42.98519302 42.97613913 ... 42.983481   42.98320964
 42.98074376]
[42.98162563 42.98519302 42.98171893 ... 42.983481   42.98320964
 42.98074376]
[42.98162563 42.98519302 42.98171893 ... 42.983481  

[43.61988818 43.6198997  43.61989013 ... 43.61989419 43.6198956
 43.61989218]
[43.61989419 43.6198997  43.61989182 ... 43.61989419 43.6198956
 43.61989218]
[43.61989419 43.6198997  43.61989182 ... 43.61989419 43.6198956
 43.61989754]
[43.61989419 43.6198997  43.61989182 ... 43.61989419 43.6198956
 43.61989754]
[43.61989419 43.61990104 43.61989448 ... 43.61989419 43.61989635
 43.61989754]
[43.61989419 43.61990104 43.61989448 ... 43.61989621 43.6198991
 43.61989785]
[43.61989419 43.61990104 43.61989735 ... 43.61989716 43.6198991
 43.61990087]
[43.61989419 43.61990104 43.61989849 ... 43.61990049 43.6198991
 43.61990087]
[43.61989785 43.61990104 43.61989849 ... 43.61990049 43.6198991
 43.61990087]
[43.61990062 43.61990104 43.61989911 ... 43.61990049 43.61990087
 43.61990306]
[43.61990062 43.61990104 43.61989911 ... 43.61990171 43.61990087
 43.61990306]
[43.61990062 43.61990104 43.61989911 ... 43.61990171 43.61990087
 43.61990306]
[43.61990165 43.61990176 43.61990246 ... 43.61990171 43.6199

[47.55323772 47.55379907 47.55447483 ... 47.5527236  47.55436164
 47.55102335]
[47.55417852 47.55379907 47.55466071 ... 47.5549503  47.55436164
 47.55466071]
[47.55423419 47.55379907 47.55466071 ... 47.5549503  47.55436164
 47.55466071]
[47.55454549 47.55379907 47.55498879 ... 47.5549503  47.55485015
 47.55466071]
[47.55507177 47.55379907 47.55504515 ... 47.5549503  47.55489259
 47.55493183]
[47.55507177 47.5546632  47.55504515 ... 47.5549503  47.55489259
 47.55493183]
[47.55565253 47.5546632  47.55524195 ... 47.55565253 47.55489259
 47.55495512]
[47.55565253 47.5546632  47.55524195 ... 47.55565253 47.55528571
 47.55560572]
[47.55565253 47.55516863 47.55524195 ... 47.55565253 47.55528571
 47.55560572]
[47.55565253 47.55516863 47.55524195 ... 47.55569675 47.55528571
 47.55560572]
[47.55565253 47.55518376 47.55555807 ... 47.55569675 47.55528571
 47.55560572]
[47.55565253 47.55563359 47.55555807 ... 47.55572696 47.55528571
 47.55560572]
[47.55565253 47.55566725 47.55579538 ... 47.55572696

...

[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796

[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796

[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796

[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]
[47.55663796 47.55663796 47.55663796 ... 47.55663796 47.55663796
 47.55663796]


(array([[0.0257634 , 0.06257391, 0.92359439, ..., 0.001     , 0.001     ,
         0.001     ],
        [0.0257634 , 0.06257391, 0.92359439, ..., 0.001     , 0.001     ,
         0.001     ],
        [0.02576341, 0.06257389, 0.9235944 , ..., 0.001     , 0.001     ,
         0.001     ],
        ...,
        [0.0257634 , 0.06257391, 0.92359439, ..., 0.001     , 0.001     ,
         0.001     ],
        [0.0257634 , 0.06257391, 0.92359439, ..., 0.001     , 0.001     ,
         0.001     ],
        [0.02576341, 0.0625739 , 0.92359439, ..., 0.001     , 0.001     ,
         0.001     ]]),
 array([47.55663796, 47.55663796, 47.55663796, ..., 47.55663796,
        47.55663796, 47.55663796]))